# visualizing termination time versus domain size
Tim Tyree<br>
6.11.2025

In [1]:
import os,sys,pandas as pd,numpy as np

In [2]:
from lib import *

# define module

In [3]:
# parse_iolines
def parse_iolines_single(input_fn, printing=False):
    n_input=-9999
    n_output=-9999
    with open(input_fn) as f:
#         trgt1='Printing Inputs...\n'
#         trgt2='Printing Outputs...\n'
        trgt1='Printing Inputs:\n'
        trgt2='Printing Outputs:\n'
        for n,line in enumerate(f):
            if trgt1 == line:
                if printing:
                    print(f'found inputs starting after line {n}')
                n_input=n
            if trgt2 == line:
                if printing:
                    print(f'found outputs starting after line {n}')
                n_output=n
    return n_input,n_output

In [4]:
def parse_outputs_single_trial(input_fn, n_output):
    """
    Example Usage:
TermTime = parse_outputs_single_trial(input_fn, n_output)
    """
    with open(input_fn) as f:
        for n,line in enumerate(f):
            if n == n_output+1:
                lin0 =line
                TermTime = eval(lin0[:-2].split('=')[-1])
                return TermTime
#         if n == n_output+1:
#             line_N = line
#         if n == n_output+2:
#             line_CollTime =line
# N_values=np.array([eval(s) for s in line_N[:-2].split(',')])
# CollTime_values=np.array([eval(s) for s in line_CollTime[:-2].split(',')])
# # CollTime_values=np.array([eval(s) for s in line_CollTime[:-1].split(',')])
# df=pd.DataFrame({
#     'N':N_values,
#     'CollTime':CollTime_values
# })
# df=df.loc[df.CollTime!=-9999]
# df.reset_index(inplace=True,drop=True)
## return df

In [5]:
# parse_outputs(input_fn, n_output=)
def parse_log_single(input_fn,printing=False):
    """
    Example Usage:
dict_data = parse_log_single(input_fn,printing=False)
    """
    n_input,n_output=parse_iolines_single(input_fn, printing=False)
    if n_input==-9999:
        if printing:
            print(f"Warning: n_input not found for input_fn={input_fn}.\nreturning None...")
        return None
    col_name_lst, col_value_lst=parse_inputs(input_fn,n_input,n_output,printing=printing)
    dict_inputs=dict(zip(col_name_lst,col_value_lst))
    TermTime = parse_outputs_single_trial(input_fn, n_output)
    dict_data = dict(dict_inputs)
    dict_data['TermTime'] = TermTime
    return dict_data

# dev

In [8]:
# fldr = f"{nb_dir}/data/osg_output/run_44"
# fldr = f"{nb_dir}/data/osg_output/run_45"
# fldr = f"{nb_dir}/data/osg_output/run_46"
fldr = f"{nb_dir}/data/osg_output/run_47"

In [9]:
assert os.path.exists(fldr)

In [10]:
dict_data_lst = []
for fn in os.listdir(fldr):
    if fn.find('.out.')!=-1:
        input_fn = os.path.join(fldr,fn)
        dict_data = parse_log_single(input_fn,printing=False)
        if dict_data is not None:
            #record
            dict_data_lst.append(dict_data)
df=pd.DataFrame(dict_data_lst); del dict_data_lst

In [11]:
# np.array([x for x in dict_data_lst if x is None])

In [12]:
# pd.DataFrame(dict_data_lst)

In [13]:
# len(dict_data_lst)

In [14]:
# input_fn

In [15]:
# !atom '/Users/timothytyree/Documents/GitHub/bgmc/python/data/osg_output/run_45/job.out.17477944.1912'

In [16]:
# dict_data = parse_log_single(input_fn,printing=True)
# dict_data

In [19]:
np.random.seed(42)
printing=True
if printing:
    print(f"LPM with Same Creation (Run #46):")
dict_lst = []
for (D,L),g in df.groupby(by=['D','L']):
    tau_values = g['TermTime'].values
    meanx,Delta_meanx,num_obs,p_normal=comp_mean_bootstrap_uncertainty(tau_values)
    printing=True
    a=g['varkappa'].max()
    chi=g['chi'].max()
    if printing:
        print(f"- {D=}, {L=}: mean tau: {meanx:.4f} +/- {Delta_meanx:.4f} seconds (N={num_obs}, {p_normal=:.4f})")
    dict_lst.append(dict(D=D,L=L,a=a,chi=chi,
                         tau=meanx,Delta_tau=Delta_meanx,
                         num_obs=num_obs,p_normal=p_normal,
                        ))

LPM with Same Creation (Run #46):
- D=0.048, L=4.5: mean tau: 9.5304 +/- 0.4177 seconds (N=2000, p_normal=0.9731)
- D=0.048, L=4.75: mean tau: 16.1093 +/- 0.6578 seconds (N=2000, p_normal=0.4783)
- D=0.048, L=5.0: mean tau: 24.9064 +/- 0.8764 seconds (N=2000, p_normal=0.6576)
- D=0.048, L=5.25: mean tau: 34.7140 +/- 0.9571 seconds (N=1999, p_normal=0.8888)
- D=0.048, L=5.625: mean tau: 47.8364 +/- 0.8291 seconds (N=2000, p_normal=0.5737)
- D=0.048, L=6.25: mean tau: 57.2602 +/- 0.4353 seconds (N=2000, p_normal=0.3579)
- D=0.42, L=5.0: mean tau: 0.5670 +/- 0.0618 seconds (N=2000, p_normal=0.0000)
- D=0.42, L=6.25: mean tau: 1.8240 +/- 0.1618 seconds (N=2000, p_normal=0.0052)
- D=0.42, L=7.5: mean tau: 7.0232 +/- 0.3804 seconds (N=2000, p_normal=0.6428)
- D=0.42, L=8.75: mean tau: 28.3362 +/- 0.9419 seconds (N=2000, p_normal=0.5027)


In [21]:
df_out = pd.DataFrame(dict_lst)
df_out

,D,L,a,chi,tau,Delta_tau,num_obs,p_normal
0,0.048,4.500,1.552,2.939433,9.530372,0.417653,2000,9.730722e-01
1,0.048,4.750,1.552,2.939433,16.109333,0.657802,2000,4.782612e-01
2,0.048,5.000,1.552,2.939433,24.906398,0.876443,2000,6.575857e-01
3,0.048,5.250,1.552,2.939433,34.713999,0.957091,1999,8.887704e-01
4,0.048,5.625,1.552,2.939433,47.836443,0.829059,2000,5.736999e-01
5,0.048,6.250,1.552,2.939433,57.260239,0.435285,2000,3.579202e-01
6,0.420,5.000,9.300,2.075269,0.566962,0.061761,2000,7.165031e-18
7,0.420,6.250,9.300,2.075269,1.823972,0.161797,2000,5.209048e-03
8,0.420,7.500,9.300,2.075269,7.023152,0.380364,2000,6.428188e-01
9,0.420,8.750,9.300,2.075269,28.336182,0.941926,2000,5.026513e-01


In [23]:
# savedir = f"{nb_dir}/data/osg_output/run_46_tau_vs_L.csv"
savedir = f"{nb_dir}/data/osg_output/run_47_tau_vs_L.csv"

In [24]:
df_out.to_csv(savedir,index=False)